In [11]:
import face_recognition
import cv2
import os
import copy
from tensorflow.keras import models
import numpy as np
from sklearn.externals import joblib
from selenium import webdriver
from PIL import ImageFont, ImageDraw, Image

In [2]:
# 학습된 모델 불러오기 (****수정하세요****)
model = joblib.load('BTS_classification_model2.pkl')

Using TensorFlow backend.


In [3]:
def my_imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None

In [16]:
# (****categoties만 수정하세요****)
categories = ["BTS 제이홉", "BTS 지민", "BTS 진", "BTS 정국",
              "BTS RM", "BTS 슈가", "BTS 뷔"]
roi_color = [(255, 255, 255), (0, 255, 255), (255, 0, 255), (255, 255, 0),
            (0, 0, 255), (0, 255, 0), (255, 0, 0)]

In [13]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
fontpath = "fonts/gulim.ttc"
font = ImageFont.truetype(fontpath, 20)

In [33]:
count=0
frame_count=0

# (****경로 수정하세요****)
cap=cv2.VideoCapture('img/bts_mv.mp4')

        
        
tmp_px=20
h_tmp_px=tmp_px//2
while True:
    is_ok, frame=cap.read()
    roi=[]
    if not is_ok:
        break
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces)!=0:
        faces_n,_=faces.shape
        for (x,y,w,h) in faces:
            roi = frame[y:y+h, x:x+w]
            roi=cv2.resize(roi,(100,100),interpolation=cv2.INTER_LINEAR)
            pred_face=roi.reshape(1,100,100,3)
            i = np.argmax(model.predict(pred_face))
            
            frame = cv2.rectangle(frame,(x-h_tmp_px,y-h_tmp_px),
                                  (x+w+h_tmp_px,y+h+h_tmp_px),roi_color[i],2)
            frame_pil = Image.fromarray(frame)
            draw = ImageDraw.Draw(frame_pil)
            draw.text((x,y+h+40), categories[i], font=font, fill=roi_color[i])
            frame = np.array(frame_pil)
            
            key = cv2.waitKey(1)
            if key == 13: # enter
                driver = webdriver.Chrome('chromedriver.exe')
                driver.implicitly_wait(1)
                driver.get('https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query='
                          + categories[i])
                break
                
            if cv2.waitKey(1) == 27: # esc
                break
    


    cv2.imshow('video',frame)
    if cv2.waitKey(1) == 27: 
        break
        
cap.release()
cv2.destroyAllWindows()